In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gfjtiti/IMUC_2020.xlsx
/kaggle/input/github-datos/README.md
/kaggle/input/github-datos/Analisis.ipynb
/kaggle/input/github-datos/Datos/calificaciones.csv
/kaggle/input/github-datos/Datos/estudiantes.csv
/kaggle/input/github-datos/Datos/carrera.csv
/kaggle/input/github-datos/Datos/escuelas.csv
/kaggle/input/filtradoo/filtrdo.csv


## Limpieza de los datos



1. Cargar los datos

In [5]:
calificaciones = pd.read_csv("/kaggle/input/github-datos/Datos/calificaciones.csv")
estudiantes = pd.read_csv("/kaggle/input/github-datos/Datos/estudiantes.csv")
carreras = pd.read_csv("/kaggle/input/github-datos/Datos/carrera.csv")
escuelas = pd.read_csv("/kaggle/input/github-datos/Datos/escuelas.csv")

### Primero eliminamos a aquellos estudiantes que su "prepa" no esta registrada como escuela

con esto se quitaron aprox. 2000-3000 filas

In [31]:
#colonias = pd.DataFrame(estudiantes["municipio"].unique(), columns=["municipio"])
#colonias

#municipio_ciudad = estudiantes.groupby('municipio')['ciudad'].unique().reset_index()
#municipio_ciudad.columns = ['municipio', 'ciudades']
#municipio_ciudad

#if estudiantes["municipio"] > 25000 && estudiantes["municipio"] < 25019:
#    estudiantes["ciudad"]


correcta = pd.merge(estudiantes, escuelas[['clave']], left_on='prepa', right_on='clave', how='inner')
correcta.sort_values('clave_alumno')



,clave_alumno,clave_carrera,sexo,prepa,colonia,municipio,ciudad,ingreso,termino,causa_salida,clave
1292,1,0,1,39,Villa de Guadalupe,25006.0,CULIACAN,2003.0,2011.0,4,39
1293,2,0,1,39,Las Mañanitas,25006.0,CULIACAN,2003.0,2013.0,4,39
1294,4,0,2,36,El Walamo,25006.0,CUALIACAN,2003.0,2061.0,4,36
1295,5,0,2,58,Las Granjas,25006.0,CULIACAN,2003.0,2011.0,4,58
1296,6,0,1,31,FIDEL VELAZQUEZ,25006.0,"ELDORADO, CLN",2003.0,2051.0,5,31
...,...,...,...,...,...,...,...,...,...,...,...
4409,11956,0,1,59,Gustavo Diaz Ordaz,25002.0,ANGOSTURA,2111.0,NaN,1,59
4410,11957,0,1,59,Gustavo Diaz Ordaz,25002.0,ANGOSTURA,2111.0,NaN,1,59
4411,11960,0,1,89,Humaya,25006.0,CULIACAN,2111.0,NaN,1,89
4412,11962,0,1,22,4 de Marzo,25006.0,CULIACAN,2111.0,NaN,1,22


### Ahora se filtran todas las calificaciones de alumnos, si la clave del alumno no esta registrada en alumnos se descartan.

In [29]:
correcta2 = pd.merge(calificaciones, estudiantes[['clave_alumno']], left_on='clave_alumno', right_on='clave_alumno', how='inner')
correcta2.sort_values('clave_alumno')

,periodo,clave_alumno,clave_materia,calificacion,tipo_calificacion
37645,2011,1,04tBt3,80.0,1
37639,2003,1,04tGt4,100.0,1
37640,2003,1,04tBt2,85.0,1
37641,2003,1,04tBt1,85.0,1
37642,2003,1,04tGt5,0.0,3
...,...,...,...,...,...
374650,2103,11853,INN1008,100.0,1
374651,2103,11853,ACA0907,80.0,1
374652,2103,11853,ACC0906,95.0,1
374653,2103,11853,ACF0901,95.0,1


### aqui filtro a aquellos estudiantes que necesitamos solamente, aquellos que siguen vigentes y que no han terminado sus estudios

In [39]:
correcta3 = estudiantes[(estudiantes['termino'].isnull()) & (estudiantes['causa_salida']==1)]
correcta3.sort_values('clave_alumno')

,clave_alumno,clave_carrera,sexo,prepa,colonia,municipio,ciudad,ingreso,termino,causa_salida
8526,1149,5,1,56,Villa del Real,25006.0,CULIACAN,2013.0,NaN,1
2561,3002,0,1,107,INF.,25006.0,CULIACAN,2043.0,NaN,1
2606,3047,0,1,107,IND.,25006.0,CULIACAN,2043.0,NaN,1
6462,3156,2,1,112,Las Granjas,25006.0,CULIACAN,2043.0,NaN,1
6487,3183,2,1,151,INF.,25006.0,CULIACAN,2043.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...
5320,11962,0,1,22,4 de Marzo,25006.0,CULIACAN,2111.0,NaN,1
11866,11963,7,1,0,NaN,NaN,NaN,2111.0,NaN,1
11263,11964,6,1,227,Universidad 94,25006.0,CULIACAN,2111.0,NaN,1
8269,11965,3,1,0,NaN,NaN,NaN,2111.0,NaN,1


In [41]:
# Convertir a minúsculas y eliminar espacios adicionales
correcta3['ciudad'] = correcta3['ciudad'].str.lower().str.strip()
correcta3['colonia'] = correcta3['colonia'].str.lower().str.strip()

correcta3
# Reemplazar abreviaturas o nombres incorrectos, ejemplo:
correcta3['ciudad'] = correcta3['ciudad'].replace({'culiacan, sin.': 'culiacan', 'culiacan': 'culiacan, sin.'})
correcta3['colonia'] = correcta3['colonia'].replace({'los angeles': 'Los Angeles', 'el walamo': 'El Walamo'})


/tmp/ipykernel_30/837248857.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correcta3['ciudad'] = correcta3['ciudad'].str.upper().str.strip()
/tmp/ipykernel_30/837248857.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correcta3['colonia'] = correcta3['colonia'].str.lower().str.strip()


,clave_alumno,clave_carrera,sexo,prepa,colonia,municipio,ciudad,ingreso,termino,causa_salida
607,3327,4,1,121,buenos,25006.0,CULIACAN,2043.0,NaN,1
648,4420,4,1,154,el walamo,25011.0,GUASAVE,2053.0,NaN,1
662,4434,4,1,127,alcanfores,25018.0,NAVOLATO,2053.0,NaN,1
668,4440,4,1,117,los angeles,25006.0,CULIACAN,2053.0,NaN,1
671,4443,4,1,142,NaN,25002.0,LA REFORMA ANG.,2053.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...
11930,11738,8,2,98,fovissste playa azul,25006.0,CULIACAN,2103.0,NaN,1
11931,11739,8,1,91,la,25006.0,CULIACAN,2103.0,NaN,1
11932,11740,8,1,71,balcones,25006.0,CULIACAN,2103.0,NaN,1
11933,11825,8,1,26,san miguel,25006.0,CULIACAN,2103.0,NaN,1


In [38]:
correcta3.to_csv('correcta3.csv', index=False)

### vuelvo a filtrar solo las calificaciones de los estudisntes que cumplen las condiciones requeridas

In [37]:
correcta4 = pd.merge(calificaciones, correcta3[['clave_alumno']], left_on='clave_alumno', right_on='clave_alumno', how='inner')
correcta4.sort_values('clave_alumno')

,periodo,clave_alumno,clave_materia,calificacion,tipo_calificacion
0,2013,1149,IFB9302,0.0,3
36,2051,1149,IFB9333,90.0,2
37,2051,1149,IFB9322,75.0,5
38,2051,1149,IFB9305,0.0,5
39,2051,1149,IFC9304,0.0,3
...,...,...,...,...,...
106772,2103,11853,INN1008,100.0,1
106773,2103,11853,ACA0907,80.0,1
106774,2103,11853,ACC0906,95.0,1
106775,2103,11853,ACF0901,95.0,1
